In [2]:
import pandas as pd
import matplotlib as plt

In [3]:
data = pd.read_csv('anomaly_data.csv')


In [4]:
for i in range(len(data)):
    if (data.loc[i, 'Pressure (psi)'] < 27 or data.loc[i, 'Pressure (psi)'] > 150 or
        data.loc[i, 'Temperature (°C)'] > 88 or
        data.loc[i, 'Oil Level (%)'] < 18 or
        data.loc[i, 'Tool RPM (rpm)'] > 2500 and
        data[i,'anomaly'] == -1):
        data.loc[i, 'Machine Status'] = 'NotRunning'

KeyError: (422, 'anomaly')

In [5]:
data.head()

,Unnamed: 0,Timestamp,Pressure (psi),Temperature (°C),Oil Flow (L/min),Vibration (mm/s),Power Status,Oil Level (%),Tool RPM (rpm),Machine Status,Current (A),scores,anomaly
0,0,17-09-2023 00:00,45.2,29.8,2.4,0.021,ON,73.5,1505,Running,10.3,0.036549,1
1,1,17-09-2023 00:00,48.2,28.8,3.4,0.031,ON,70.5,1805,Running,11.3,0.085057,1
2,2,17-09-2023 00:01,45.1,29.9,2.5,0.022,ON,73.3,1508,Running,10.4,0.040864,1
3,3,17-09-2023 00:01,48.1,28.9,3.5,0.032,ON,70.3,1808,Running,11.4,0.092148,1
4,4,17-09-2023 00:02,45.3,30.0,2.6,0.023,ON,73.2,1512,Running,10.5,0.046241,1


In [ ]:
data.to_csv('data_aug.csv', index=False) 

In [ ]:
data['Vibration (mm/s)'].median()

0.6

In [ ]:
plt.figure(figsize=(20,50))
ax=data['time_cycles'].plot(kind='barh',width=0.8, stacked=True,align='center')

AttributeError: module 'matplotlib' has no attribute 'figure'